In [22]:
import pandas as pd
import spacy
import fastai
from fastai.basic_data import *
from fastai.datasets import *
from fastai.text.data import *
from fastai.text.learner import *
from fastai.text import *
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

In [2]:
from ulmfit import *
from ulmfit.pretrain_lm import *
from ulmfit.__main__ import *

In [3]:
print(fastai.__version__)
print(spacy.__version__)

1.0.52
2.1.1


In [4]:
MODEL_SUBPATH = "models/f30k"
bs=40

In [5]:
path_lm = Path('data/ulmfit/wiki/id-2')
path_cls = Path('data/ulmfit/tmp/id-2')

In [6]:
#path_lm.ls()

In [7]:
path_cls.ls()

[PosixPath('data/ulmfit/tmp/id-2/test1-err.log'),
 PosixPath('data/ulmfit/tmp/id-2/test2-err.log'),
 PosixPath('data/ulmfit/tmp/id-2/prepare_wiki-1.log'),
 PosixPath('data/ulmfit/tmp/id-2/prepare_wiki-1-error.log'),
 PosixPath('data/ulmfit/tmp/id-2/id.test.csv'),
 PosixPath('data/ulmfit/tmp/id-2/id.train.csv'),
 PosixPath('data/ulmfit/tmp/id-2/id.unsup.csv'),
 PosixPath('data/ulmfit/tmp/id-2/models')]

In [8]:
#data_lm = load_data(path_lm/f'{MODEL_SUBPATH}', 'lm', bs=bs)

In [9]:
ulmfit = ULMFiT()

In [10]:
ulmfit_cls = ulmfit.cls(dataset_path=path_cls, base_lm_path=path_lm/f'{MODEL_SUBPATH}/lstm_orig.m', 
                       name='orig', lang='id')

current directory is : /home/cahya/Work/MachineLearning/FastAI/FastAI-Course
Max vocab: 30000
Cache dir: /mnt/mldata/data/LM/ulmfit/tmp/id-2/models/f30k
Model dir: /mnt/mldata/data/LM/ulmfit/tmp/id-2/models/f30k/lstm_orig.m


In [15]:
learner, data_lm, data_cls, metric = ulmfit_cls.train(20, bs=bs, num_cls_epochs=4, lr_sched='1cycle', 
                                                      label_smoothing_eps=0.1, use_test_for_validation=False)

Loss and accuracy using (cls_best): [0.24403837, tensor(0.9451), tensor(0.9468)] [0.26475385, tensor(0.9385), tensor(0.9400)]


/home/cahya/Work/miniconda3/envs/fastai-1.0/lib/python3.7/site-packages/fastai/metrics.py:179: UserWarning: average=`binary` was selected for a non binary case. Value for average has now been set to `macro` instead.
  warn("average=`binary` was selected for a non binary case. Value for average has now been set to `macro` instead.")


In [13]:
# learner, data_lm, data_cls, metric = ulmfit_cls.validate_cls()

In [16]:
metric

{'val_loss': 0.26475385,
 'val_accuracy': 0.9384828805923462,
 'tst_loss': 0.24403837,
 'tst_accuracy': 0.9450686573982239}

In [26]:
learner.predict("ekonomi di indonesia sudah mulai maju")

(Category 0, tensor(0), tensor([0.9251, 0.0275, 0.0155, 0.0319]))

In [27]:
learner.predict("pssi menang sepakbola di lapangan bola")

(Category 3, tensor(3), tensor([0.0903, 0.0481, 0.0499, 0.8117]))

In [28]:
learner.predict("Badai matahari juga bisa menyebabkan gangguan pada sistem pengendali orientasi ketinggian stasiun antariksa di orbit bum")

(Category 2, tensor(2), tensor([0.0259, 0.0326, 0.9145, 0.0269]))

In [29]:
learner.predict("Kanselir Jerman Angela Merkel bertemu dengan Presiden AS George W. Bush, Kamis, untuk mengadakan pembicaraan yang diperkirakan dipusatkan pada upaya guna memperkuat hubungan trans-Atlantik")

(Category 1, tensor(1), tensor([0.0338, 0.8992, 0.0502, 0.0168]))

In [17]:
predictions = learner.get_preds(ds_type=DatasetType.Valid, ordered=True)

In [18]:
y_pred = np.argmax(predictions[0].numpy(), axis=1)

In [19]:
y_true = predictions[1].numpy()

In [20]:
print(len(y_pred), len(y_true))

2403 2403


In [21]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

In [27]:
metrics = precision_recall_fscore_support(y_true, y_pred, average='macro')

In [28]:
metrics

(0.9465822560032131, 0.9469338053516916, 0.9466812832378351, None)

In [29]:
confusion_matrix(y_true, y_pred)

array([[655,  12,  22,   1],
       [  8, 622,  29,   9],
       [ 17,  16, 580,   4],
       [  0,   9,   5, 414]])

In [30]:
learner.validate()

/home/cahya/Work/miniconda3/envs/fastai-1.0/lib/python3.7/site-packages/fastai/metrics.py:179: UserWarning: average=`binary` was selected for a non binary case. Value for average has now been set to `macro` instead.
  warn("average=`binary` was selected for a non binary case. Value for average has now been set to `macro` instead.")


[0.24403837, tensor(0.9451), tensor(0.9468)]